In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re
import geopandas as gpd
import pickle
import yaml
import pickle

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

In [7]:
data = pd.read_parquet("../../data/02_interim/sample_11_02_all_features_high_radius.parquet")

In [ ]:
data = pd.read_parquet("../data/sample_11_02_all_features_high_radius.parquet")

In [3]:
with open("../04_deviation_detection/random_forest_model.pkl", 'rb') as fin:
    model = pickle.load(fin)
    
with open("../04_deviation_detection/features.yaml", 'r') as fin:
    features = yaml.safe_load(fin)

In [11]:
features

['accelerometer_x_corr',
 'accelerometer_x_diff_corr',
 'accelerometer_y_corr',
 'accelerometer_y_diff_corr',
 'accelerometer_z_corr',
 'accelerometer_z_diff_corr',
 'x_metres_corr',
 'x_diff_metres_corr',
 'y_metres_corr',
 'y_diff_metres_corr',
 'height_corr',
 'bearing_corr',
 'speed_corr',
 'height_diff_corr',
 'bearing_diff_corr',
 'speed_diff_corr',
 'speed_mean_abs_diff',
 'speed_mean_diff',
 'speed_std_diff',
 'distance_mean',
 'distance_std']

In [8]:
data.loc[data["dev_id_1"].str[-1] == data["dev_id_2"].str[-1], 'target'] = 1

In [9]:
data['target'] = data['target'].fillna(0)

In [32]:
data['predicted_proba'] = model.predict_proba(data[features].fillna(-999))[:, 1]

In [33]:
data['thresh'] = 0.416

In [35]:
data['label'] = (data['predicted_proba'] >= data['thresh']).astype(int)

In [39]:
data['target'].value_counts()

0.0    426
1.0     57
Name: target, dtype: int64

In [42]:
data.shape[0]

483

In [43]:
477 / 483

0.9875776397515528

In [44]:
data.to_csv("../../data/05_analytical_report_new/02_anomaly_detection.csv", index=False)

In [41]:
(data['target'] == data['label']).sum()

477

In [37]:
data["label"].value_counts()

0    430
1     53
Name: label, dtype: int64

In [29]:
data['target']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
478    0.0
479    0.0
480    0.0
481    0.0
482    0.0
Name: target, Length: 483, dtype: float64

In [24]:
data["dev_id_1"].unique()

array(['D4909E26-0469-4045-B426-211006C38915',
       'A1303E56-0163-1015-B156-511006C38315'], dtype=object)

In [22]:
(model.predict_proba(data[features].fillna(-999))[:, 1] > 0.416).sum()

53

In [ ]:
data["predict_proba"]

In [6]:
data["id_1_last_geometry"].isna().sum()

6552